In [46]:
import pandas as pd
import numpy as np
import sqlalchemy as sal

In [48]:
# Read the data
df = pd.read_csv("Walmart.csv")

In [49]:
df.head(5)

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [50]:
# Drop duplicates
df = df.drop_duplicates()



In [51]:
# Clean price data - remove $ and convert to float
df["unit_price"] = df["unit_price"].str.replace("$", "").astype(float)


In [53]:
# Calculate total price
df["total_price"] = df["unit_price"] * df["quantity"]


In [56]:
# Convert date to datetime
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%y")

In [57]:
# Convert time string to proper time format
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time


In [58]:
df.dtypes

invoice_id                 int64
Branch                    object
City                      object
category                  object
unit_price               float64
quantity                 float64
date              datetime64[ns]
time                      object
payment_method            object
rating                   float64
profit_margin            float64
total_price              float64
dtype: object

In [59]:
# Convert column names to lowercase
df.columns = df.columns.str.lower()
# Drop rows with null values
df = df.dropna()
print("Number of rows after dropping nulls:", len(df))


Number of rows after dropping nulls: 9969


In [61]:
df.head(5)

,invoice_id,branch,city,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total_price
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,2019-01-05,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,2019-03-08,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,2019-03-03,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,2019-01-27,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,2019-02-08,10:37:00,Ewallet,5.3,0.48,604.17


In [60]:
# PostgreSQL connection
username = "jasim"
database =  "wallmart_sales"
password = "ab123693"
host = "localhost"
port = "5432"

# Create SQLAlchemy engine
engine = sal.create_engine(
    f"postgresql+pg8000://{username}:{password}@{host}:{port}/{database}"
)

# Define data types for PostgreSQL
dtype_mapping = {
    "invoice_id": sal.Integer,
    "branch": sal.String,
    "city": sal.String,
    "category": sal.String,
    "unit_price": sal.Float,
    "quantity": sal.Float,
    "date": sal.Date,
    "time": sal.Time,
    "payment_method": sal.String,
    "rating": sal.Float,
    "profit_margin": sal.Float,
    "total_price": sal.Float,
}

# Upload to PostgreSQL
try:
    with engine.connect() as conn:
        df.to_sql(
            "sales_data",
            con=conn,
            index=False,
            if_exists="replace",
            dtype=dtype_mapping,
        )
        print("Data loaded successfully with proper date and time formats")
except Exception as e:
    print(f"Error occurred: {str(e)}")


Data loaded successfully with proper date and time formats
